In [21]:
import pandas as pd
import numpy as np
import re
import os
from sklearn.cluster import KMeans

In [22]:
#reading data from csv files
user_feat = pd.read_csv('user_features.csv')



print user_feat

spice_feat = pd.read_csv('spice_features.csv')

print spice_feat
#food user rating table
food_user = pd.DataFrame(np.random.randint(low=0, high=2, size=(len(spice_feat),len(user_feat))),columns=user_feat.user_guid)
food_user.index  =  spice_feat['food_id']
#print food_user.head()

    Age Sex     Location  NonVeg  user_guid
0    30   F         U.P.       1        500
1    23   M          J&K       1        501
2    87   M  Maharashtra       2        502
3    44   M      Haryana       2        503
4    20   F      Haryana       1        504
5    50   M  Maharashtra       1        505
6    27   M       Punjab       2        506
7    27   F  South India       2        507
8     7   M       Punjab       2        508
9    54   M       Punjab       1        509
10   65   M         U.P.       0        510
11   88   M      Haryana       1        511
12   43   M       Punjab       0        512
13   14   M          J&K       2        513
14   18   M      Haryana       1        514
15   82   F         U.P.       2        515
16   86   F          J&K       1        516
17   74   M       Punjab       1        517
18   20   F  South India       2        518
19   38   M      Haryana       0        519
20   56   M         U.P.       2        520
21   61   F       Punjab       2

In [23]:
#Data Normalisation

def nor_age(age):
    
    if(age<=10):
        return 1
    elif(age>10 and age <=18):
        return 2
    elif(age>18 and age <=25):
        return 3
    elif(age>25 and age <=35):
        return 4
    elif(age>35 and age <=40):
        return 5
    elif(age>40 and age <=50):
        return 6
    elif(age>50 and age<=60):
        return 7
    else:
        return 8
    
    

s_dict = {"M":1,"F":5 }
l_dict = {'J&K':2, 'Haryana':5, 'Punjab':6, 'U.P.':8, 'Maharashtra':10, 'South India':15}
user_feat['Age'] = user_feat['Age'].apply(nor_age)
user_feat  = user_feat.replace({"Sex":s_dict})
user_feat  = user_feat.replace({"Location":l_dict})


In [24]:
print "enter the user id"

input_user_id = input()



enter the user id
555


In [26]:
kmeans = KMeans(n_clusters = 10,random_state=0).fit(user_feat)
user_feat['label'] = kmeans.labels_
#print user_feat

index = np.where(user_feat['user_guid']==input_user_id)[0]
#print index[0]
user_label = user_feat.loc[index[0],'label']
#print user_label
user = user_feat.sort_values(['label','Location'],ascending=[1,0])
user = user.drop_duplicates(subset=['Location','label'])

user = user.groupby('label')['Location'].nlargest(3)
print user.head()
loc = user[user_label].tolist()
print loc



label    
0      99    15
       94    10
       92     8
1      34    10
       38     8
Name: Location, dtype: int64
[6, 5, 2]


In [27]:
print "enter veg/non-beg"

veggy = input()

print "enter user required calories of current meal"

upper_calorie = input()

enter veg/non-beg
2
enter user required calories of current meal
200


In [28]:
newspice_feat = spice_feat.set_index('food_id')
spice = newspice_feat[newspice_feat['Location'].isin(loc)]
#print spice
spice = spice[newspice_feat['NonVeg']==veggy]
#print spice
s_spice =  spice.sort_values('count',ascending=[0])
s_index = food_user.index[food_user[input_user_id]==0]
#print s_index

new_spices = s_spice[s_spice.index.isin(s_index)]
print new_spices
new_spices = new_spices[new_spices['calorie']<upper_calorie]
print new_spices

         spice1  spice2  spice3  spice4  count  Location  NonVeg  calorie
food_id                                                                  
60070         1       0       0       0      4         2       2      376
60092         1       0       1       1      2         5       2      136
60010         1       1       1       0      2         6       2      221
60026         1       1       0       1      2         2       2      339
60047         0       0       1       1      2         5       2      253
60083         1       1       1       0      1         2       2      247
60060         0       0       1       0      0         5       2      313
60004         0       1       1       0      0         5       2      492
60025         1       0       1       0      0         6       2      157
         spice1  spice2  spice3  spice4  count  Location  NonVeg  calorie
food_id                                                                  
60092         1       0       1       

/home/demonicode/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [29]:
#integrating randomness

from random import randint
#print(randint(0, 9))

x = randint(0,9)
print x

7


In [30]:
recommend = []

if(x==5):
    print ('random')  # to test
    index_to_append = spice_feat.index[randint(0,len(spice_feat))]
    recommend.append(spice_feat.loc[index_to_append,'food_id'])
    
    for i in range(min(2,len(new_spices))):
        recommend.append(new_spices.index[i])
    
    if(len(recommend)==0):
        other_state_spice = spice_feat[spice_feat['Location'].isin(loc)==False]
    
        other_state_spice = other_state_spice.sort('count',ascending= [0])
        new_other_state = other_state_spice[other_state_spice.index.isin(s_index)]

        for i in range(min(2,len(new_other_state))):
            recommend.append(new_other_state.index[i])
            
else:
    print ('not random')
    for i in range(min(3,len(new_spices))):
        recommend.append(new_spices.index[i])
    
    if(len(recommend)==0):
        other_state_spice = spice_feat[spice_feat['Location'].isin(loc)==False]
    
        other_state_spice = other_state_spice.sort('count',ascending= [0])
        new_other_state = other_state_spice[other_state_spice.index.isin(s_index)]

        for i in range(min(3,len(new_other_state))):
            recommend.append(new_other_state.index[i])

    
print recommend    

not random
[60092, 60025]


In [34]:
spice = np.array(spice_feat.iloc[:,0:4])
#print spice
user_df = pd.DataFrame(np.random.randint(low=-1, high=2, size=(len(spice_feat),len(user_feat))),columns=user_feat.user_guid,index=spice_feat.food_id)

print user_df
user = np.array(user_df)
#user tells rating and spice tells if ingrediant is there or not.

global spice
global user

user_guid  500  501  502  503  504  505  506  507  508  509 ...   590  591  \
food_id                                                     ...              
60000        1   -1   -1    1    0    0   -1   -1    1   -1 ...     0   -1   
60001        1   -1    1    0    0    1    0    0   -1    1 ...     0    1   
60002        0    1    0    0    1   -1    0    1    0   -1 ...     1    0   
60003        1    0   -1    1    1    0    1   -1    0   -1 ...    -1    0   
60004       -1    1   -1    0   -1    1   -1   -1    0    0 ...    -1    0   
60005        0    0   -1    1    1   -1    0    0   -1    0 ...     1   -1   
60006        1    1    0   -1   -1   -1    1   -1    1    1 ...    -1   -1   
60007        1   -1   -1   -1    1    1    1   -1    0   -1 ...     1    0   
60008       -1   -1   -1    0    0    0    0    1    0   -1 ...    -1   -1   
60009        1    1    1   -1   -1   -1    0    1   -1    0 ...     0    0   
60010        0    0   -1   -1    0    1    1    0    0    1 ... 

In [35]:
# we have user_table which will give rating to a particular dish
# content based recommeder system
# we have 2 tables with us

#one is food - spices table and another is food-user rating table
#we will use tf-idf approach
#matrix of spice will contain only of 1 and 0's so need to normalize it

#normalization along the row
def spice_norm():
    
    num_attr = np.sum(spice,axis=1)
    doc_vec = []
    for i in range(num_attr.shape[0]):
        if(num_attr[i] != 0): 
            doc_vec.append(np.sqrt(spice[i,:]*1.0/num_attr[i]))
        else:
            doc_vec.append(spice[i,:])
    return np.array(doc_vec)      
    
    
    
def attr_norm():
    
    num_attr = np.sum(spice,axis=0)
    
    doc_vec = []
    for i in range(num_attr.shape[0]):
        if(num_attr[i] != 0):
            doc_vec.append(spice[:,i]*1.0 / num_attr[i])
        else:
            doc_vec.append(spice[:,i])
    return np.array(doc_vec)

def user_v(spice_mat):
    user_vec = []
    
    
    #print user[:,0]
    #print spice_mat[:,0]
    
    for i in range(user.shape[1]):
        user_vec.append([np.dot(user[:,i],spice_mat[:,j]) for j in range(spice_mat.shape[1])])
    
    print (user_vec)
    return np.array(user_vec)
    
def weight_predict(spice_mat):
        
    user_vec = user_v(spice_mat)
    
    preds = []
    #print spice_mat
    for i in range(spice_mat.shape[0]):
        preds.append([np.dot(spice_mat[i,:],user_vec[j,:]) for j in range(user_vec.shape[0])])

    return np.array(preds)
    
    
def idf_weight_pred():
    spice_mat = spice_norm()
    attr_wght = attr_norm()
    #print attr_wght
    weighted_spice = []
    
    for i in range(spice_mat.shape[0]):
        weighted_spice.append([np.dot(spice_mat[i,:],attr_wght[:,j]) for j in range(attr_wght.shape[0])])
        
    preds  = weight_predict(spice_mat = np.array(weighted_spice))     
    
    return np.array(preds)
    


preds = idf_weight_pred()

#print preds.shape

[[0.22582909642454094, 0.10570703193390982, 0.10570703193390982, 0.21225730833544679], [0.13355374050644098, -0.0068747610853769278, -0.0068747610853769278, 0.10096218928021004], [0.024666213214979374, 0.10434457500249721, 0.10434457500249721, 0.0049606277494654225], [-0.042153035802976663, 0.0037176072409024, 0.0037176072409024, -0.088316375118301621], [0.076342200701927876, 0.043464418612182087, 0.043464418612182087, -0.034393969589748538], [-0.025783087239817759, 0.038893416202771758, 0.038893416202771758, -0.098902532323613679], [-0.15102818912148991, 0.011610491852088713, 0.011610491852088713, -0.0092318920647457767], [-0.051887777035543793, -0.034157685436059959, -0.034157685436059959, -0.13482223942596333], [-0.046358024281773075, -0.11014982092854156, -0.11014982092854156, -0.01149326648727645], [-0.020519458222209956, -0.028861501272920316, -0.028861501272920316, -0.027769978525166869], [0.025311089513307836, 0.1259944201299375, 0.1259944201299375, 0.046181258152317287], [0.35

In [36]:
outputdf = pd.DataFrame(data = preds,index = newspice_feat.index ,columns = user_df.columns)
#print user_df
#print outputdf
s_index = food_user.index[food_user[input_user_id]==0]
#print s_index

output = outputdf[outputdf.index.isin(s_index)]
#print user_df.columns
newoutput = output[input_user_id].sort_values(ascending = False)
#print newoutput
food_to_recommend = list(newoutput[0:3].index)

print food_to_recommend

[60042, 60077, 60032]
